# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Storage solutions for Big Data using Neo4j** </center>
---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Create SparkSession 


In [2]:
from pyspark.sql import SparkSession

# Hint for homework: .config("spark.jars.packages", "groupId:artifactId:version") \

spark = SparkSession.builder \
    .appName("Spark-Nero4j-Integration") \
    .master("spark://8776010e8f6a:7077") \
    .config("spark.jars", "/home/jovyan/notebooks/jars/neo4j-connector-apache-spark_2.13-5.3.5_for_spark_3.jar") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

25/03/26 21:18:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


#### Load ecommerce Dataset

In [3]:

from team_name.spark_utils import SparkUtils

columns_info = [("product", "string"),
                ("price", "double"),
                ("quantity", "integer"),
                ("discount", "float"),
                ("customer_first_name", "string"),
                ("order_date", "date")]

schema = SparkUtils.generate_schema(columns_info)

# Create DataFrame
ecommerce_df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/e_commerce_dataset/")

ecommerce_df.printSchema()

root
 |-- product: string (nullable = true)
 |-- price: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- discount: float (nullable = true)
 |-- customer_first_name: string (nullable = true)
 |-- order_date: date (nullable = true)



#### Write DF in Neo4j

In [5]:
neo4j_url = "bolt://neo4j-iteso:7687"
neo4j_user = "neo4j"
neo4j_password = "123456789"

ecommerce_df.write \
    .format("org.neo4j.spark.DataSource") \
    .mode("Overwrite") \
    .option("url", neo4j_url) \
    .option("authentication.type", "basic") \
    .option("authentication.basic.username", neo4j_user) \
    .option("authentication.basic.password", neo4j_password) \
    .option("labels", ":Product") \
    .option("node.keys", "product") \
    .save()

print("Done")

Done


In [6]:
# Stop the SparkContext
sc.stop()

#### Create a DF from Neo4j

In [7]:
spark = SparkSession.builder \
    .appName("Spark-Nero4j-Integration") \
    .master("spark://8776010e8f6a:7077") \
    .config("spark.jars", "/home/jovyan/notebooks/jars/neo4j-connector-apache-spark_2.13-5.3.5_for_spark_3.jar") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

##### Set Database connection properties 

In [8]:
df = spark.read \
    .format("org.neo4j.spark.DataSource") \
    .option("url", neo4j_url) \
    .option("authentication.type", "basic") \
    .option("authentication.basic.username", neo4j_user) \
    .option("authentication.basic.password", neo4j_password) \
    .option("labels", ":Product") \
    .load()

df.printSchema()
df.show(5, False)

25/03/26 21:30:29 WARN SchemaService: Switching to query schema resolution


root
 |-- <id>: long (nullable = false)
 |-- <labels>: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quantity: long (nullable = true)
 |-- product: string (nullable = true)
 |-- price: double (nullable = true)
 |-- order_date: date (nullable = true)
 |-- discount: double (nullable = true)
 |-- customer_first_name: string (nullable = true)



+----+---------+--------+-------------------------------------+-----+----------+--------+-------------------+
|<id>|<labels> |quantity|product                              |price|order_date|discount|customer_first_name|
+----+---------+--------+-------------------------------------+-----+----------+--------+-------------------+
|0   |[Product]|10      |Wireless Presenter Remote            |82.29|2024-11-04|44.0    |Sarah              |
|1   |[Product]|15      |Electric Standing Desk               |7.65 |2024-04-19|44.0    |Jasmine            |
|2   |[Product]|166     |Smart Thermostat                     |87.94|2022-07-02|20.0    |Cheryl             |
|3   |[Product]|220     |Electric Toothbrush with UV Sanitizer|22.78|2024-07-04|4.0     |Jorge              |
|4   |[Product]|7       |Smart Fitness Tracker Watch          |25.74|2024-05-03|1.0     |Renee              |
+----+---------+--------+-------------------------------------+-----+----------+--------+-------------------+
only showi

In [9]:
sc.stop()